In [1]:
import psycopg2
import re

In [2]:
def connect():
    def executeScriptsFromFile(filename):
        with open(filename, 'r') as sql_file:
            cur.execute(sql_file.read())
                
    def transformCheckConstraintsFromFile(filename_read, filename_trigger, filename_save):
        # get all check constraints
        fd = open(filename_read, 'r')
        sqlFile = fd.read()
        fd.close()
        str_list = re.findall(r"(?<=CHECK).*", sqlFile)
        
        # TODO: is there a less hard-coded way to find the table name?
        table_name = re.findall(r"(?<=CREATE TABLE ).*", sqlFile)
        table_name = table_name[0].split()[0]

        # Copy create_table.sql to a new sql file, and remove all CHECK sections
        # Find all occurences of "CHECK"
        index_check = [m.start() for m in re.finditer('CHECK', sqlFile)]
        
        # Find occurences of ")" corresponding to each "CHECK"
        index_right_par = []
        for i in range(len(index_check)):
            end_pos = -1 if i == len(index_check) - 1 else index_check[i+1]
            index = [m.start() for m in re.finditer('\)', sqlFile[index_check[i]: end_pos])][0]    
            index_right_par.append(index + index_check[i])

        # Remove substrings "CHECK...)" from end to start (so that index doesn't change)
        for i in range(len(index_check)-1, -1, -1):
            sqlFile = sqlFile[0:index_check[i]-1] + sqlFile[index_right_par[i]+1:]
            
        cur.execute(sqlFile)
        
        # TODO: Now we assume the format of salary numeric CHECK (salary > 0)
        # But here are more possibilities:
        # salary numeric CHECK (salary > 0 AND salary < 5000)
        # salary numeric CONSTRAINT check_salary_limit CHECK (salary > 0 AND salary < 5000)
        # any more?
        
        constraints_list = []
        for i in range(len(str_list)):
            constraint = str_list[i]
            constraint = constraint[constraint.index('(')+1:constraint.index(')')].split()
            constraints_list.append(constraint)

        # transform each check constraint to a trigger
        fd = open(filename_trigger, 'r')
        skeleton = fd.read()
        fd.close()
        triggers_list = []
        for constraint in constraints_list:
            # if the right side is a column name, add "new." in front of it
            command  = """SELECT EXISTS(SELECT 1
                        FROM information_schema.columns 
                        WHERE table_name='{}' AND column_name={});""".format(table_name, 
                        constraint[2] if constraint[2][0]=="\'" else "\'" + constraint[2] + "\'")
            cur.execute(command)
            results = cur.fetchall()
            trigger = skeleton.format(table_name, constraint[0], 
                                table_name, 
                                table_name, constraint[0],
                                table_name, constraint[0], 
                                constraint[0], 
                                constraint[1],
                                constraint[2] if results[0][0] == False else "new."+constraint[2], 
                                constraint[0], table_name, 
                                table_name, constraint[0], table_name, 
                                table_name, constraint[0])
            triggers_list.append(trigger)

        for trigger in triggers_list:
            sqlFile = sqlFile + "\n" + "\n" + trigger
       
        # insert all triggers to the new file
        with open(filename_save, 'w') as file:
            file.write(sqlFile)

    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(
            host="localhost",
            database="cs5421",
            user="postgres",
            password="common2021")# Change the database name & credentials accordingly!

        # create a cursor
        cur = conn.cursor()
        
        # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
        # execute commands from the .sql file
#         executeScriptsFromFile("create_table.sql")
        
        # TODO: multiple create table in one file?
        
        # For create table...
        # transform the check constraints and save into new .sql file
        transformCheckConstraintsFromFile("create_table.sql", "function_trigger_skeleton.txt", "create_table_transformed.sql")
        executeScriptsFromFile("create_table_transformed.sql")

        # TODO: For alter table... need to handle drop check
        # TODO: create table & alter table in same file?
        
        # Run test queries
#         executeScriptsFromFile("query1.sql")
        
        # TODO: compare performance of check vs.trigger
        
        # commit the changes
        conn.commit()
        
        # close the communication with the PostgreSQL
        cur.close()
    
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


if __name__ == '__main__':
    connect()
    


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 15.2 (Ubuntu 15.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0, 64-bit',)
Database connection closed.


Some possibilities to handle:

salary numeric CHECK (salary > 0) ==> default name employees_salary_check, trigger needs to have this name! <table_name>_<column_name>_check, , function has name of name of verify_<table_name>_<column_name>_check

salary numeric CONSTRAINT salary_limit CHECK (salary < 5000) ==> trigger has the given name, function has name of verify_<trigger_name> 

DROP CONSTRAINT employees_salary_check ==> has check in the name, remove trigger and function if exist

DROP CONSTRAINT salary_limit ==> may not be a check constraint, remove trigger and function if exist